In [1]:
"""IMPORTANT: MUST RUN CELLS IN ORDER!!
    2022 New Zealand International
"""
import os
import time

import boto3

from lifter_api import LifterAPI

In [10]:
# authenticate

URL = None
# URL = "http://localhost:8000"

auth_token = os.getenv("LOCAL_API_TOKEN")
api = LifterAPI(url=URL, auth_token=auth_token)
if URL != "http://localhost:8000":
    auth_token = os.getenv("API_TOKEN")
    api = LifterAPI(url="https://api.lifter.shivan.xyz", auth_token=auth_token)
    print("connected LIVE")
else:
    print("connected locally")

connected LIVE


In [12]:
# Create competition and the sessions
# This will also check if the competition already exists by name, and also sessions by name as well.
def create_competition(date_start, date_end, competition_name, location):
    # search for a competition
    current_competitions = {}
    next_page = "page=1"
    while next_page:
        competitions = api.competitions(page=int(next_page.split("page")[-1].replace("=", "")))
        for competition in competitions['results']:
            current_competitions[competition['competition_name']] = competition['reference_id']
        next_page = competitions['next']

    current_competition_names = list(current_competitions.keys())
    
    if competition_name not in current_competition_names:
        print(f"{competition_name} created.")
        return api.create_competition(date_start=date_start,
                            date_end=date_end,
                            location=location,
                            competition_name=competition_name)
    print(f"ERROR: {competition_name} already exists!")
    return api.get_competition(competition_id=current_competitions[competition_name])

def create_competition_and_sessions(competition_data: dict[str: str], sessions_data: list[dict[str, str]]):
    

    def create_session(competition_id, session_datetime, referee_first, referee_second, referee_third, technical_controller, jury, marshall, announcer, timekeeper):
        # search for sessions
        sessions = api.sessions(competition_id=competition_id)
        current_sessions = {}
        # FIXME: unable to go through pagination
        for session in sessions['results']:
            current_sessions[session['session_datetime']] = session['reference_id']
        
        current_session_datetimes = list(current_sessions.keys())
        
        if session_datetime not in current_session_datetimes:
            print(f"Session at {session_datetime} created.")
            return api.create_session(
                competition_id=competition_id,
                session_datetime=session_datetime,
                referee_first=referee_first,
                referee_second=referee_second,
                referee_third=referee_third,
                technical_controller=technical_controller,
                jury=jury,
                marshall=marshall,
                announcer=announcer,
                timekeeper=timekeeper,
            )
        print(f"ERROR: Session at {session_datetime} already exists.")
        return api.get_session(competition_id=competition_id, session_id=current_sessions[session_datetime])

    new_competition = create_competition(
        date_start=competition_data['date_start'],
        date_end=competition_data['date_end'],
        location=competition_data['location'],
        competition_name=competition_data['competition_name']
    )

    for session_data in sessions_data:
        new_session = create_session(
            competition_id=new_competition['reference_id'],
            session_datetime=session_data['session_datetime'],
            referee_first=session_data['referee_first'],
            referee_second=session_data['referee_second'],
            referee_third=session_data['referee_third'],
            technical_controller=session_data['technical_controller'],
            jury=session_data['jury'],
            marshall=session_data['marshall'],
            announcer=session_data['announcer'],
            timekeeper=session_data['timekeeper'],
        )

In [13]:
competition_data = {
    'date_start': "2022-02-26",
    'date_end': "2022-02-27",
    'location': "Kolmar, Auckland",
    'competition_name': "New Zealand International",
}

sessions_data = [ 
    {
        'session_datetime': "2022-02-26T10:00:00+13:00",
        'referee_first': "Jason Fanning",
        'referee_second': "Odette Smith",
        'referee_third': "Paule Poulin",
        'technical_controller': "Alex Stewart",
        'jury': "Barbara Grieve, Keisha-Dean Soffe, Alan Tano",
        'marshall': "Karl Duncan, Karl Davies",
        'announcer': "Empty",
        'timekeeper': "Kirsty Walker",
    }, 
    {
        'session_datetime': "2022-02-26T13:30:00+13:00",
        'referee_first': "Paule Poulin",
        'referee_second': "Karl Duncan",
        'referee_third': "Jason Fanning",
        'technical_controller': "Andy Jameson",
        'jury': "Barbara Grieve, Keisha-Dean Soffe, Alan Tano",
        'marshall': "Odette Smith, Karl Davies",
        'announcer': "Empty",
        'timekeeper': "Alex Stewart",
    }, 
    {
        'session_datetime': "2022-02-27T10:00:00+13:00",
        'referee_first': "Jason Fanning",
        'referee_second': "Odette Smith",
        'referee_third': "Paule Poulin",
        'technical_controller': "Alex Jameson",
        'jury': "Barbara Grieve, Keisha-Sean Soffee",
        'marshall': "Andy Jameson, Kirsty Walker",
        'announcer': "Empty",
        'timekeeper': "Genice Paulley-Beazley",
    },
    {
        'session_datetime': "2022-02-27T13:00:00+13:00",
        'referee_first': "Paule Poulin",
        'referee_second': "Karl Duncan",
        'referee_third': "Jason Fanning",
        'technical_controller': "Richie Wong",
        'jury': "Barbara Grieve, Keisha-Dean Soffe, Alan Tano",
        'marshall': "Alex Stewart",
        'announcer': "Empty",
        'timekeeper': "Genice Paulley-Beazley",
    }
]

create_competition_and_sessions(competition_data=competition_data, sessions_data=sessions_data)

New Zealand International created.
Session at 2022-02-26T10:00:00+13:00 created.
Session at 2022-02-26T13:30:00+13:00 created.
Session at 2022-02-27T10:00:00+13:00 created.
Session at 2022-02-27T13:00:00+13:00 created.


In [14]:
# create athletes

def create_athlete(first_name: str, last_name: str, yearborn: int):
    first_name = first_name
    last_name = last_name
    if api.find_athlete(search=(first_name + " " + last_name))["count"] == 0:
        create = api.create_athlete(
            first_name=first_name, last_name=last_name, yearborn=yearborn
        )
        print(f"athlete created {create['first_name']} {create['last_name']}")
    else:
        print("athlete already exists")

In [28]:
# create lift

competition_id = "rbEp8Ev"

def create_lift(competition_id: str, session_id: str, lifts_data: list[dict[str: str|int]]):
    for lift in lifts_data:
        print(lift)
        first_name = lift["athlete_name"].split(" ")[0]
        last_name = lift["athlete_name"].split(" ")[1]
        athlete = api.find_athlete(search=(first_name + " " + last_name))
        print(athlete['results'][0]['reference_id'])
        if athlete['count'] == 1:
            api.create_lift(
                competition_id=competition_id,
                session_id=session_id,
                athlete_id=athlete['results'][0]['reference_id'],
                snatch_first=lift['snatch_first'],
                snatch_first_weight=lift['snatch_first_weight'],
                snatch_second=lift['snatch_second'],
                snatch_second_weight=lift['snatch_second_weight'],
                snatch_third=lift['snatch_third'],
                snatch_third_weight=lift['snatch_third_weight'],
                cnj_first=lift['cnj_first'],
                cnj_first_weight=lift['cnj_first_weight'],
                cnj_second=lift['cnj_second'],
                cnj_second_weight=lift['cnj_second_weight'],
                cnj_third=lift['cnj_third'],
                cnj_third_weight=lift['cnj_third_weight'],
                bodyweight=lift['bodyweight'],
                weight_category=lift['weight_category'],
                team=lift['team'],
                lottery_number=lift['lottery_number'],
                )
            print("Lift created successfully")
        elif athlete['count'] > 1:
            print("Multiple athlete with the name query exist")
        else:
            print(f"No athlete of the name, {lift['athlete_name']} this name exist")

In [16]:
def upload_pdf_s3(file_path, bucket, object_name=None):
    if object_name is None:
        object_name = os.path.basename(file_path)
    s3_client = boto3.client('s3')
    s3_client.upload_file(file_path, bucket, object_name)

def analyse_pdf(file_path, bucket, object_name=None):
    if object_name is None:
        object_name = os.path.basename(file_path)
    client = boto3.client('textract', region_name='ap-southeast-2')
    response = client.start_document_analysis(
        DocumentLocation={
            'S3Object': {
                'Bucket': bucket,
                'Name': object_name}},
            FeatureTypes=["TABLES"]
    )
    return response

In [17]:
FILE_PATH = "./data/2022/2022.NZ_International.pdf"
BUCKET = "reading-optom-notes"

upload_pdf_s3(file_path=FILE_PATH, bucket=BUCKET)

In [18]:
response = analyse_pdf(file_path=FILE_PATH, bucket=BUCKET)

In [19]:
client = boto3.client('textract')
result = client.get_document_analysis(JobId=response['JobId'])
result

{'JobStatus': 'IN_PROGRESS',
 'AnalyzeDocumentModelVersion': '1.0',
 'ResponseMetadata': {'RequestId': '3a99ad99-1ec1-475d-95ca-6c67477efb59',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '3a99ad99-1ec1-475d-95ca-6c67477efb59',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '63',
   'date': 'Tue, 12 Apr 2022 10:09:51 GMT'},
  'RetryAttempts': 0}}

In [20]:
def get_analysis():
    client = boto3.client('textract')
    result = client.get_document_analysis(JobId=response['JobId'])
    print(result['JobStatus'])
    if result["JobStatus"] == "IN_PROGRESS":
        time.sleep(5)
        result = get_analysis()
    return result

result = get_analysis()

results = [result]

while result.get("NextToken", None):
    result = client.get_document_analysis(JobId=response['JobId'],NextToken=result['NextToken'])
    results.append(result)

IN_PROGRESS
IN_PROGRESS
IN_PROGRESS
SUCCEEDED


In [21]:
# get table ids
ids_from_tables = []
for result in results:
    for blocks in result['Blocks']:
        if blocks["BlockType"] == "TABLE":
            ids_from_tables.extend(blocks["Relationships"][0]["Ids"])

In [22]:
# get blocks assocated with the table ids
blocks_related_table = []
for result in results:
    for block in result['Blocks']:
        for i in ids_from_tables:
            if block["Id"] == i and block.get("Relationships", None):
                blocks_related_table.append(block)

In [23]:
# turn blocks into dictionary
blockDictionary = {}
for result in results:
    for block in result['Blocks']:
        blockDictionary[block["Id"]] = block

In [24]:
# extract relationships from table ids
table_text = []
for block_related_table in blocks_related_table:
    table_text.append([blockDictionary[block_id]["Text"] for block_id in block_related_table["Relationships"][0]["Ids"]])

In [105]:
def give_lift_status(weight: str):
    weight = weight.replace(' ','')
    if weight.isdigit():
        return ("LIFT", weight)
    elif (weight[0], weight[-1]) == ("(",")"):
        weight = weight.replace('(','')
        weight = weight.replace(')','')
        return ("NOLIFT", weight)
    elif weight == '-':
        return ("DNA", 0)

def give_weight_category(weight_category: str):
    weight_category = ''.join(weight_category)
    weight_category = weight_category.replace(' ', '')
    weight_category = weight_category.replace('196','96')
    weight_category = weight_category.replace('>87', '87+')
    weight_category = weight_category.replace('>109', '109+')
    weight_category = weight_category.replace('F', 'W')
    return weight_category

def parse_lift(lift: list[list[str]]):
    country = "NZ"
    if lift[6] == ["South", "Africa"]:
        country = "ZA"
    if lift[6] == ["Tonga"]:
        country = "TO"
    if lift[6] == ["Cook", "Islands"]:
        country = "CK"
    if lift[6] == ["Niue"]:
        country = "NU"
    if lift[6] == ["Fiji"]:
        country = "FJ"
    if lift[6] == ["Guest"]:
        country = "G"

    lift_data = { 
        "athlete_name": f"{lift[3][0]} {lift[2][0]}",
        "snatch_first": give_lift_status(lift[8][0])[0],
        "snatch_first_weight": int(give_lift_status(lift[8][0])[1]),
        "snatch_second": give_lift_status(lift[9][0])[0],
        "snatch_second_weight": int(give_lift_status(lift[9][0])[1]),
        "snatch_third": give_lift_status(lift[10][0])[0],
        "snatch_third_weight": int(give_lift_status(lift[10][0])[1]),
        "cnj_first": give_lift_status(lift[12][0])[0],
        "cnj_first_weight": int(give_lift_status(lift[12][0])[1]),
        "cnj_second": give_lift_status(lift[13][0])[0],
        "cnj_second_weight": int(give_lift_status(lift[13][0])[1]),
        "cnj_third": give_lift_status(lift[14][0])[0],
        "cnj_third_weight": int(give_lift_status(lift[14][0])[1]),
        "bodyweight": float(lift[5][0]),
        "weight_category": give_weight_category(lift[4]),
        "team": country,
        "lottery_number": int(lift[1][0]),
    }
    return lift_data 

In [106]:
# add lifts and create athlete

def create_athlete_lift(start: int, interval: int, table_text: list, competition_id: str, session_id: str):

    lift_info = table_text[start:start+interval]
    for i in lift_info:
        print(i, end=" ")
        
    valid = "N"
    if input(f"{lift_info}") == "Y":
    # if True:
        create_athlete(
            first_name=lift_info[3][0],
            last_name=lift_info[2][0],
            yearborn=lift_info[7][0],
        )
        create_lift(competition_id=competition_id, session_id=session_id, lifts_data=[parse_lift(lift_info)])
        
    print('\n')
    print(start+interval)

In [30]:
# Elizabeth Granger

start = 26
interval = 20
session_id = "9AKzrK6"

create_athlete_lift(
    start=start,
    interval=interval, 
    table_text=table_text,
    competition_id=competition_id,
    session_id=session_id
    )

['1'] ['3'] ['Granger'] ['Elizabeth'] ['F 59'] ['59.00'] ['New', 'Zealand'] ['1991'] ['73'] ['75'] ['(77)'] ['75'] ['88'] ['92'] ['94'] ['94'] ['169'] ['1'] ['230.822'] ['325.189'] athlete already exists
{'athlete_name': 'Elizabeth Granger', 'snatch_first': 'LIFT', 'snatch_first_weight': 73, 'snatch_second': 'LIFT', 'snatch_second_weight': 75, 'snatch_third': 'NOLIFT', 'snatch_third_weight': 77, 'cnj_first': 'LIFT', 'cnj_first_weight': 88, 'cnj_second': 'LIFT', 'cnj_second_weight': 92, 'cnj_third': 'LIFT', 'cnj_third_weight': 94, 'bodyweight': 59.0, 'weight_category': 'W59', 'team': 'NZ', 'lottery_number': 3}
zbrJDNG
Lift created successfully


46


In [33]:
# Mona Pretorius

start = 46
interval = 20

session_id = "9AKzrK6"

create_athlete_lift(
    start=start,
    interval=interval, 
    table_text=table_text,
    competition_id=competition_id,
    session_id=session_id
    )

['2'] ['8'] ['Pretorius'] ['Mona'] ['F 59'] ['59.00'] ['South', 'Africa'] ['1988'] ['68'] ['(71)'] ['71'] ['71'] ['88'] ['91'] ['(94)'] ['91'] ['162'] ['2'] ['221.261'] ['282.557'] athlete created Mona Pretorius
{'athlete_name': 'Mona Pretorius', 'snatch_first': 'LIFT', 'snatch_first_weight': 68, 'snatch_second': 'NOLIFT', 'snatch_second_weight': 71, 'snatch_third': 'LIFT', 'snatch_third_weight': 71, 'cnj_first': 'LIFT', 'cnj_first_weight': 88, 'cnj_second': 'LIFT', 'cnj_second_weight': 91, 'cnj_third': 'NOLIFT', 'cnj_third_weight': 94, 'bodyweight': 59.0, 'weight_category': 'W59', 'team': 'ZA', 'lottery_number': 8}
W8A21rz
Lift created successfully


66


In [34]:

# Riana Froger

start = 66
interval = 19

session_id = "9AKzrK6"

create_athlete_lift(
    start=start,
    interval=interval, 
    table_text=table_text,
    competition_id=competition_id,
    session_id=session_id
    )

['3'] ['11'] ['Froger'] ['Riana'] ['F 59'] ['58.40'] ['New', 'Zealand'] ['1996'] ['(65)'] ['(65)'] ['(65)'] ['-'] ['88'] ['91'] ['94'] ['94'] ['-'] ['-'] ['-'] athlete created Riana Froger
{'athlete_name': 'Riana Froger', 'snatch_first': 'NOLIFT', 'snatch_first_weight': 65, 'snatch_second': 'NOLIFT', 'snatch_second_weight': 65, 'snatch_third': 'NOLIFT', 'snatch_third_weight': 65, 'cnj_first': 'LIFT', 'cnj_first_weight': 88, 'cnj_second': 'LIFT', 'cnj_second_weight': 91, 'cnj_third': 'LIFT', 'cnj_third_weight': 94, 'bodyweight': 58.4, 'weight_category': 'W59', 'team': 'NZ', 'lottery_number': 11}
4RrXmAO
Lift created successfully


85


In [35]:
# Selemaia Olivia

start = 86
interval = 20

session_id = "9AKzrK6"

create_athlete_lift(
    start=start,
    interval=interval, 
    table_text=table_text,
    competition_id=competition_id,
    session_id=session_id
    )

['4'] ['19'] ['Selemaia'] ['Olivia'] ['F 64'] ['62.50'] ['New', 'Zealand'] ['2006'] ['80'] ['(83)'] ['83'] ['83'] ['100'] ['(105)'] ['(106)'] ['100'] ['183'] ['2'] ['241.007'] ['526.361'] athlete created Olivia Selemaia
{'athlete_name': 'Olivia Selemaia', 'snatch_first': 'LIFT', 'snatch_first_weight': 80, 'snatch_second': 'NOLIFT', 'snatch_second_weight': 83, 'snatch_third': 'LIFT', 'snatch_third_weight': 83, 'cnj_first': 'LIFT', 'cnj_first_weight': 100, 'cnj_second': 'NOLIFT', 'cnj_second_weight': 105, 'cnj_third': 'NOLIFT', 'cnj_third_weight': 106, 'bodyweight': 62.5, 'weight_category': 'W64', 'team': 'NZ', 'lottery_number': 19}
D9YZKrd
Lift created successfully


106


In [36]:
# Emma McIntyre

start = 106
interval = 20

session_id = "9AKzrK6"

create_athlete_lift(
    start=start,
    interval=interval, 
    table_text=table_text,
    competition_id=competition_id,
    session_id=session_id
    )

['5'] ['24'] ['McIntyre'] ['Emma'] ['F 64'] ['63.60'] ['New', 'Zealand'] ['1991'] ['82'] ['85'] ['87'] ['87'] ['101'] ['(104)'] ['104'] ['104'] ['191'] ['1'] ['248.896'] ['387.922'] athlete created Emma McIntyre
{'athlete_name': 'Emma McIntyre', 'snatch_first': 'LIFT', 'snatch_first_weight': 82, 'snatch_second': 'LIFT', 'snatch_second_weight': 85, 'snatch_third': 'LIFT', 'snatch_third_weight': 87, 'cnj_first': 'LIFT', 'cnj_first_weight': 101, 'cnj_second': 'NOLIFT', 'cnj_second_weight': 104, 'cnj_third': 'LIFT', 'cnj_third_weight': 104, 'bodyweight': 63.6, 'weight_category': 'W64', 'team': 'NZ', 'lottery_number': 24}
84YaoAm
Lift created successfully


126


In [37]:
# Georgia Wedd

start = 126
interval = 20

session_id = "9AKzrK6"

create_athlete_lift(
    start=start,
    interval=interval, 
    table_text=table_text,
    competition_id=competition_id,
    session_id=session_id
    )

['6'] ['33'] ['Wedd'] ['Georgia'] ['F 64'] ['63.60'] ['New', 'Zealand'] ['1992'] ['64'] ['68'] ['(72)'] ['68'] ['(90)'] ['90'] ['(94)'] ['90'] ['158'] ['3'] ['205.893'] ['206.584'] athlete created Georgia Wedd
{'athlete_name': 'Georgia Wedd', 'snatch_first': 'LIFT', 'snatch_first_weight': 64, 'snatch_second': 'LIFT', 'snatch_second_weight': 68, 'snatch_third': 'NOLIFT', 'snatch_third_weight': 72, 'cnj_first': 'NOLIFT', 'cnj_first_weight': 90, 'cnj_second': 'LIFT', 'cnj_second_weight': 90, 'cnj_third': 'NOLIFT', 'cnj_third_weight': 94, 'bodyweight': 63.6, 'weight_category': 'W64', 'team': 'NZ', 'lottery_number': 33}
qkYLGNE
Lift created successfully


146


In [38]:
# Emma Wright

start = 147
interval = 20

session_id = "9AKzrK6"

create_athlete_lift(
    start=start,
    interval=interval, 
    table_text=table_text,
    competition_id=competition_id,
    session_id=session_id
    )

['7'] ['17'] ['Wright'] ['Emma'] ['F 71'] ['69.40'] ['New', 'Zealand'] ['1991'] ['77'] ['81'] ['84'] ['84'] ['96'] ['100'] ['(104)'] ['100'] ['184'] ['4'] ['228.126'] ['290.313'] athlete created Emma Wright
{'athlete_name': 'Emma Wright', 'snatch_first': 'LIFT', 'snatch_first_weight': 77, 'snatch_second': 'LIFT', 'snatch_second_weight': 81, 'snatch_third': 'LIFT', 'snatch_third_weight': 84, 'cnj_first': 'LIFT', 'cnj_first_weight': 96, 'cnj_second': 'LIFT', 'cnj_second_weight': 100, 'cnj_third': 'NOLIFT', 'cnj_third_weight': 104, 'bodyweight': 69.4, 'weight_category': 'W71', 'team': 'NZ', 'lottery_number': 17}
vVr8pr7
Lift created successfully


167


In [39]:
# Samara Wright

start = 167
interval = 20


session_id = "9AKzrK6"

create_athlete_lift(
    start=start,
    interval=interval, 
    table_text=table_text,
    competition_id=competition_id,
    session_id=session_id
    )

['8'] ['23'] ['Wright'] ['Samara'] ['F 71'] ['70.10'] ['New', 'Zealand'] ['2001'] ['(79)'] ['79'] ['(82)'] ['79'] ['(104)'] ['104'] ['107'] ['107'] ['186'] ['2'] ['229.365'] ['300.928'] athlete created Samara Wright
{'athlete_name': 'Samara Wright', 'snatch_first': 'NOLIFT', 'snatch_first_weight': 79, 'snatch_second': 'LIFT', 'snatch_second_weight': 79, 'snatch_third': 'NOLIFT', 'snatch_third_weight': 82, 'cnj_first': 'NOLIFT', 'cnj_first_weight': 104, 'cnj_second': 'LIFT', 'cnj_second_weight': 104, 'cnj_third': 'LIFT', 'cnj_third_weight': 107, 'bodyweight': 70.1, 'weight_category': 'W71', 'team': 'NZ', 'lottery_number': 23}
51N1Kr0
Lift created successfully


187


In [41]:
# Rachel Duncan

start = 187
interval = 20

session_id = "9AKzrK6"

create_athlete_lift(
    start=start,
    interval=interval, 
    table_text=table_text,
    competition_id=competition_id,
    session_id=session_id
    )

['9'] ['32'] ['Duncan'] ['Rachel'] ['F 71'] ['66.70'] ['New', 'Zealand'] ['2003'] ['75'] ['(80)'] ['82'] ['82'] ['98'] ['103'] ['(106)'] ['103'] ['185'] ['3'] ['234.468'] ['358.180'] athlete created Rachel Duncan
{'athlete_name': 'Rachel Duncan', 'snatch_first': 'LIFT', 'snatch_first_weight': 75, 'snatch_second': 'NOLIFT', 'snatch_second_weight': 80, 'snatch_third': 'LIFT', 'snatch_third_weight': 82, 'cnj_first': 'LIFT', 'cnj_first_weight': 98, 'cnj_second': 'LIFT', 'cnj_second_weight': 103, 'cnj_third': 'NOLIFT', 'cnj_third_weight': 106, 'bodyweight': 66.7, 'weight_category': 'W71', 'team': 'NZ', 'lottery_number': 32}
MmN4MYx
Lift created successfully


207


In [42]:
# Megan Signal

start = 207
interval = 20

session_id = "9AKzrK6"

create_athlete_lift(
    start=start,
    interval=interval, 
    table_text=table_text,
    competition_id=competition_id,
    session_id=session_id
    )

['10'] ['36'] ['Signal'] ['Megan'] ['F 71'] ['70.90'] ['New', 'Zealand'] ['1990'] ['80'] ['83'] ['(85)'] ['83'] ['103'] ['(108)'] ['108'] ['108'] ['191'] ['1'] ['234.118'] ['328.649'] athlete created Megan Signal
{'athlete_name': 'Megan Signal', 'snatch_first': 'LIFT', 'snatch_first_weight': 80, 'snatch_second': 'LIFT', 'snatch_second_weight': 83, 'snatch_third': 'NOLIFT', 'snatch_third_weight': 85, 'cnj_first': 'LIFT', 'cnj_first_weight': 103, 'cnj_second': 'NOLIFT', 'cnj_second_weight': 108, 'cnj_third': 'LIFT', 'cnj_third_weight': 108, 'bodyweight': 70.9, 'weight_category': 'W71', 'team': 'NZ', 'lottery_number': 36}
7wNKmrG
Lift created successfully


227


In [47]:
# Vester Villalon

start = 253
interval = 20
session_id = "BPlBblk"

create_athlete_lift(
    start=start,
    interval=interval, 
    table_text=table_text,
    competition_id=competition_id,
    session_id=session_id
    )

['1'] ['37'] ['Villalon'] ['Vester'] ['M 73'] ['72.90'] ['New', 'Zealand'] ['1991'] ['122'] ['125'] ['128'] ['128'] ['149'] ['154'] ['159'] ['159'] ['287'] ['1'] ['369.285'] ['458.228'] athlete created Vester Villalon
{'athlete_name': 'Vester Villalon', 'snatch_first': 'LIFT', 'snatch_first_weight': 122, 'snatch_second': 'LIFT', 'snatch_second_weight': 125, 'snatch_third': 'LIFT', 'snatch_third_weight': 128, 'cnj_first': 'LIFT', 'cnj_first_weight': 149, 'cnj_second': 'LIFT', 'cnj_second_weight': 154, 'cnj_third': 'LIFT', 'cnj_third_weight': 159, 'bodyweight': 72.9, 'weight_category': 'M73', 'team': 'NZ', 'lottery_number': 37}
JvAEnNE
Lift created successfully


273


In [52]:
# Curran Power

start = 274
interval = 20
session_id = "BPlBblk"

create_athlete_lift(
    start=start,
    interval=interval, 
    table_text=table_text,
    competition_id=competition_id,
    session_id=session_id
    )

['2'] ['5'] ['Power'] ['Curran'] ['M', '81'] ['79.50'] ['New', 'Zealand'] ['1997'] ['116'] ['121'] ['125'] ['125'] ['136'] ['142'] ['(147)'] ['142'] ['267'] ['2'] ['327.685'] ['315.104'] athlete already exists
{'athlete_name': 'Curran Power', 'snatch_first': 'LIFT', 'snatch_first_weight': 116, 'snatch_second': 'LIFT', 'snatch_second_weight': 121, 'snatch_third': 'LIFT', 'snatch_third_weight': 125, 'cnj_first': 'LIFT', 'cnj_first_weight': 136, 'cnj_second': 'LIFT', 'cnj_second_weight': 142, 'cnj_third': 'NOLIFT', 'cnj_third_weight': 147, 'bodyweight': 79.5, 'weight_category': 'M81', 'team': 'NZ', 'lottery_number': 5}
a8YVpAG
Lift created successfully


294


In [53]:
# Johan Roberts
start = 294
interval = 20
session_id = "BPlBblk"

create_athlete_lift(
    start=start,
    interval=interval, 
    table_text=table_text,
    competition_id=competition_id,
    session_id=session_id
    )

['3'] ['16'] ['Roberts'] ['Johan'] ['M 81'] ['80.10'] ['New', 'Zealand'] ['2004'] ['115'] ['120'] ['(124)'] ['120'] ['155'] ['(160)'] ['(160)'] ['155'] ['275'] ['1'] ['336.200'] ['366.546'] athlete created Johan Roberts
{'athlete_name': 'Johan Roberts', 'snatch_first': 'LIFT', 'snatch_first_weight': 115, 'snatch_second': 'LIFT', 'snatch_second_weight': 120, 'snatch_third': 'NOLIFT', 'snatch_third_weight': 124, 'cnj_first': 'LIFT', 'cnj_first_weight': 155, 'cnj_second': 'NOLIFT', 'cnj_second_weight': 160, 'cnj_third': 'NOLIFT', 'cnj_third_weight': 160, 'bodyweight': 80.1, 'weight_category': 'M81', 'team': 'NZ', 'lottery_number': 16}
wLN0Jr0
Lift created successfully


314


In [54]:
start = 314
interval = 20
session_id = "BPlBblk"

create_athlete_lift(
    start=start,
    interval=interval, 
    table_text=table_text,
    competition_id=competition_id,
    session_id=session_id
    )

['4'] ['30'] ['Tura'] ['Jerome'] ['M 81'] ['80.20'] ['Cook', 'Islands'] ['1991'] ['100'] ['105'] ['110'] ['110'] ['135'] ['140'] ['(148)'] ['140'] ['250'] ['3'] ['305.441'] ['253.246'] athlete created Jerome Tura
{'athlete_name': 'Jerome Tura', 'snatch_first': 'LIFT', 'snatch_first_weight': 100, 'snatch_second': 'LIFT', 'snatch_second_weight': 105, 'snatch_third': 'LIFT', 'snatch_third_weight': 110, 'cnj_first': 'LIFT', 'cnj_first_weight': 135, 'cnj_second': 'LIFT', 'cnj_second_weight': 140, 'cnj_third': 'NOLIFT', 'cnj_third_weight': 148, 'bodyweight': 80.2, 'weight_category': 'M81', 'team': 'CK', 'lottery_number': 30}
blrMXYy
Lift created successfully


334


In [59]:
start = 335
interval = 20
session_id = "BPlBblk"

create_athlete_lift(
    start=start,
    interval=interval, 
    table_text=table_text,
    competition_id=competition_id,
    session_id=session_id
    )

['6'] ['1'] ['Sekone-Fraser'] ['Douglas'] ['M', '1 96'] ['95.00'] ['New', 'Zealand'] ['1991'] ['(132)'] ['136'] ['(141)'] ['136'] ['163'] ['(167)'] ['(167)'] ['163'] ['299'] ['1'] ['338.147'] ['333.858'] athlete already exists
{'athlete_name': 'Douglas Sekone-Fraser', 'snatch_first': 'NOLIFT', 'snatch_first_weight': 132, 'snatch_second': 'LIFT', 'snatch_second_weight': 136, 'snatch_third': 'NOLIFT', 'snatch_third_weight': 141, 'cnj_first': 'LIFT', 'cnj_first_weight': 163, 'cnj_second': 'NOLIFT', 'cnj_second_weight': 167, 'cnj_third': 'NOLIFT', 'cnj_third_weight': 167, 'bodyweight': 95.0, 'weight_category': 'M96', 'team': 'NZ', 'lottery_number': 1}
d2rdkYw
Lift created successfully


355


In [61]:
start = 355
interval = 20
session_id = "BPlBblk"

create_athlete_lift(
    start=start,
    interval=interval, 
    table_text=table_text,
    competition_id=competition_id,
    session_id=session_id
    )

['7'] ['4'] ['Klenner'] ['Colin'] ['M 96'] ['93.60'] ['New', 'Zealand'] ['1987'] ['115'] ['(120)'] ['(125)'] ['115'] ['150'] ['155'] ['(160)'] ['155'] ['270'] ['4'] ['307.195'] ['237.890'] athlete created Colin Klenner
{'athlete_name': 'Colin Klenner', 'snatch_first': 'LIFT', 'snatch_first_weight': 115, 'snatch_second': 'NOLIFT', 'snatch_second_weight': 120, 'snatch_third': 'NOLIFT', 'snatch_third_weight': 125, 'cnj_first': 'LIFT', 'cnj_first_weight': 150, 'cnj_second': 'LIFT', 'cnj_second_weight': 155, 'cnj_third': 'NOLIFT', 'cnj_third_weight': 160, 'bodyweight': 93.6, 'weight_category': 'M96', 'team': 'NZ', 'lottery_number': 4}
2KYB4Am
Lift created successfully


375


In [62]:
start = 375
interval = 20
session_id = "BPlBblk"

create_athlete_lift(
    start=start,
    interval=interval, 
    table_text=table_text,
    competition_id=competition_id,
    session_id=session_id
    )

['8'] ['15'] ['Funaki'] ['Uaealesi'] ['M 96'] ['95.10'] ['Tonga'] ['1998'] ['125'] ['(130)'] ['(130)'] ['125'] ['(165)'] ['(165)'] ['(166)'] ['-'] ['-'] ['-'] ['9'] ['18'] athlete created Uaealesi Funaki
{'athlete_name': 'Uaealesi Funaki', 'snatch_first': 'LIFT', 'snatch_first_weight': 125, 'snatch_second': 'NOLIFT', 'snatch_second_weight': 130, 'snatch_third': 'NOLIFT', 'snatch_third_weight': 130, 'cnj_first': 'NOLIFT', 'cnj_first_weight': 165, 'cnj_second': 'NOLIFT', 'cnj_second_weight': 165, 'cnj_third': 'NOLIFT', 'cnj_third_weight': 166, 'bodyweight': 95.1, 'weight_category': 'M96', 'team': 'TO', 'lottery_number': 15}
WlNpMYv
Lift created successfully


395


In [64]:
start = 393
interval = 20
session_id = "BPlBblk"

create_athlete_lift(
    start=start,
    interval=interval, 
    table_text=table_text,
    competition_id=competition_id,
    session_id=session_id
    )

['9'] ['18'] ['Albert'] ['Xavier'] ['M 96'] ['91.70'] ['New', 'Zealand'] ['1998'] ['125'] ['(130)'] ['(133)'] ['125'] ['145'] ['149'] ['153'] ['153'] ['278'] ['3'] ['319.015'] ['262.121'] athlete created Xavier Albert
{'athlete_name': 'Xavier Albert', 'snatch_first': 'LIFT', 'snatch_first_weight': 125, 'snatch_second': 'NOLIFT', 'snatch_second_weight': 130, 'snatch_third': 'NOLIFT', 'snatch_third_weight': 133, 'cnj_first': 'LIFT', 'cnj_first_weight': 145, 'cnj_second': 'LIFT', 'cnj_second_weight': 149, 'cnj_third': 'LIFT', 'cnj_third_weight': 153, 'bodyweight': 91.7, 'weight_category': 'M96', 'team': 'NZ', 'lottery_number': 18}
4nAnLY0
Lift created successfully


413


In [66]:
start = 413
interval = 19
session_id = "BPlBblk"

create_athlete_lift(
    start=start,
    interval=interval, 
    table_text=table_text,
    competition_id=competition_id,
    session_id=session_id
    )

['5'] ['21'] ['McTaggart'] ['Cameron'] ['M 96'] ['86.60'] ['New', 'Zealand'] ['1997'] ['(138)'] ['(138)'] ['(138)'] ['-'] ['165'] ['171'] ['(176)'] ['171'] ['-'] ['-'] ['-'] athlete created Cameron McTaggart
{'athlete_name': 'Cameron McTaggart', 'snatch_first': 'NOLIFT', 'snatch_first_weight': 138, 'snatch_second': 'NOLIFT', 'snatch_second_weight': 138, 'snatch_third': 'NOLIFT', 'snatch_third_weight': 138, 'cnj_first': 'LIFT', 'cnj_first_weight': 165, 'cnj_second': 'LIFT', 'cnj_second_weight': 171, 'cnj_third': 'NOLIFT', 'cnj_third_weight': 176, 'bodyweight': 86.6, 'weight_category': 'M96', 'team': 'NZ', 'lottery_number': 21}
DWNvLry
Lift created successfully


432


In [68]:
start = 432
interval = 20
session_id = "BPlBblk"

create_athlete_lift(
    start=start,
    interval=interval, 
    table_text=table_text,
    competition_id=competition_id,
    session_id=session_id
    )

['10'] ['28'] ['Dombroski'] ['Ethan'] ['M 96'] ['94.90'] ['New', 'Zealand'] ['2001'] ['110'] ['118'] ['(123)'] ['118'] ['150'] ['(158)'] ['(161)'] ['150'] ['268'] ['5'] ['303.216'] ['232.086'] athlete created Ethan Dombroski
{'athlete_name': 'Ethan Dombroski', 'snatch_first': 'LIFT', 'snatch_first_weight': 110, 'snatch_second': 'LIFT', 'snatch_second_weight': 118, 'snatch_third': 'NOLIFT', 'snatch_third_weight': 123, 'cnj_first': 'LIFT', 'cnj_first_weight': 150, 'cnj_second': 'NOLIFT', 'cnj_second_weight': 158, 'cnj_third': 'NOLIFT', 'cnj_third_weight': 161, 'bodyweight': 94.9, 'weight_category': 'M96', 'team': 'NZ', 'lottery_number': 28}
q8ro7rX
Lift created successfully


452


In [69]:
start = 452
interval = 20
session_id = "BPlBblk"

create_athlete_lift(
    start=start,
    interval=interval, 
    table_text=table_text,
    competition_id=competition_id,
    session_id=session_id
    )

['11'] ['35'] ['Urgert'] ['Cam'] ['M', '96'] ['93.40'] ['New', 'Zealand'] ['1993'] ['135'] ['(139)'] ['(140)'] ['135'] ['(160)'] ['161'] ['(166)'] ['161'] ['296'] ['2'] ['337.073'] ['322.860'] athlete created Cam Urgert
{'athlete_name': 'Cam Urgert', 'snatch_first': 'LIFT', 'snatch_first_weight': 135, 'snatch_second': 'NOLIFT', 'snatch_second_weight': 139, 'snatch_third': 'NOLIFT', 'snatch_third_weight': 140, 'cnj_first': 'NOLIFT', 'cnj_first_weight': 160, 'cnj_second': 'LIFT', 'cnj_second_weight': 161, 'cnj_third': 'NOLIFT', 'cnj_third_weight': 166, 'bodyweight': 93.4, 'weight_category': 'M96', 'team': 'NZ', 'lottery_number': 35}
KeAg9YM
Lift created successfully


472


In [72]:
start = 498
interval = 20
session_id = "arDNGgQ"

create_athlete_lift(
    start=start,
    interval=interval, 
    table_text=table_text,
    competition_id=competition_id,
    session_id=session_id
    )

['1'] ['10'] ['Baarspul'] ['Renee'] ['F 76'] ['75.50'] ['New', 'Zealand'] ['1986'] ['83'] ['88'] ['(91)'] ['88'] ['101'] ['(105)'] ['(106)'] ['101'] ['189'] ['1'] ['224.428'] ['291.213'] athlete created Renee Baarspul
{'athlete_name': 'Renee Baarspul', 'snatch_first': 'LIFT', 'snatch_first_weight': 83, 'snatch_second': 'LIFT', 'snatch_second_weight': 88, 'snatch_third': 'NOLIFT', 'snatch_third_weight': 91, 'cnj_first': 'LIFT', 'cnj_first_weight': 101, 'cnj_second': 'NOLIFT', 'cnj_second_weight': 105, 'cnj_third': 'NOLIFT', 'cnj_third_weight': 106, 'bodyweight': 75.5, 'weight_category': 'W76', 'team': 'NZ', 'lottery_number': 10}
BzNGwNV
Lift created successfully


518


In [75]:
start = 518
interval = 19
session_id = "arDNGgQ"

create_athlete_lift(
    start=start,
    interval=interval, 
    table_text=table_text,
    competition_id=competition_id,
    session_id=session_id
    )

['2'] ['25'] ['Jones'] ['Medea'] ['F 76'] ['75.00'] ['New', 'Zealand'] ['2003'] ['85'] ['(89)'] ['89'] ['89'] ['(97)'] ['(97)'] ['(97)'] ['-'] ['-'] ['-'] ['-'] athlete created Medea Jones
{'athlete_name': 'Medea Jones', 'snatch_first': 'LIFT', 'snatch_first_weight': 85, 'snatch_second': 'NOLIFT', 'snatch_second_weight': 89, 'snatch_third': 'LIFT', 'snatch_third_weight': 89, 'cnj_first': 'NOLIFT', 'cnj_first_weight': 97, 'cnj_second': 'NOLIFT', 'cnj_second_weight': 97, 'cnj_third': 'NOLIFT', 'cnj_third_weight': 97, 'bodyweight': 75.0, 'weight_category': 'W76', 'team': 'NZ', 'lottery_number': 25}
eXAyDrm
Lift created successfully


537


In [76]:
start = 537
interval = 20
session_id = "arDNGgQ"

create_athlete_lift(
    start=start,
    interval=interval, 
    table_text=table_text,
    competition_id=competition_id,
    session_id=session_id
    )

['3'] ['27'] ['Hansen'] ['Sam'] ['F 76'] ['75.50'] ['New', 'Zealand'] ['1999'] ['82'] ['(85)'] ['(85)'] ['82'] ['106'] ['(110)'] ['(113)'] ['106'] ['188'] ['2'] ['223.241'] ['286.126'] athlete created Sam Hansen
{'athlete_name': 'Sam Hansen', 'snatch_first': 'LIFT', 'snatch_first_weight': 82, 'snatch_second': 'NOLIFT', 'snatch_second_weight': 85, 'snatch_third': 'NOLIFT', 'snatch_third_weight': 85, 'cnj_first': 'LIFT', 'cnj_first_weight': 106, 'cnj_second': 'NOLIFT', 'cnj_second_weight': 110, 'cnj_third': 'NOLIFT', 'cnj_third_weight': 113, 'bodyweight': 75.5, 'weight_category': 'W76', 'team': 'NZ', 'lottery_number': 27}
MXYReNm
Lift created successfully


557


In [77]:
start = 557
interval = 20
session_id = "arDNGgQ"

create_athlete_lift(
    start=start,
    interval=interval, 
    table_text=table_text,
    competition_id=competition_id,
    session_id=session_id
    )

['4'] ['31'] ['Drain'] ['Holly'] ['F 76'] ['75.60'] ['New', 'Zealand'] ['1998'] ['80'] ['(84)'] ['(85)'] ['80'] ['92'] ['96'] ['(100)'] ['96'] ['176'] ['3'] ['208.858'] ['229.826'] athlete created Holly Drain
{'athlete_name': 'Holly Drain', 'snatch_first': 'LIFT', 'snatch_first_weight': 80, 'snatch_second': 'NOLIFT', 'snatch_second_weight': 84, 'snatch_third': 'NOLIFT', 'snatch_third_weight': 85, 'cnj_first': 'LIFT', 'cnj_first_weight': 92, 'cnj_second': 'LIFT', 'cnj_second_weight': 96, 'cnj_third': 'NOLIFT', 'cnj_third_weight': 100, 'bodyweight': 75.6, 'weight_category': 'W76', 'team': 'NZ', 'lottery_number': 31}
R0r6brE
Lift created successfully


577


In [86]:
start = 578
interval = 20
session_id = "arDNGgQ"

create_athlete_lift(
    start=start,
    interval=interval, 
    table_text=table_text,
    competition_id=competition_id,
    session_id=session_id
    )

['5'] ['7'] ['Andrews-Nahu'] ['Kanah'] ['F 87'] ['86.00'] ['New', 'Zealand'] ['2001'] ['95'] ['98'] ['100'] ['100'] ['115'] ['118'] ['122'] ['122'] ['222'] ['2'] ['248.959'] ['393.318'] athlete already exists
{'athlete_name': 'Kanah Andrews-Nahu', 'snatch_first': 'LIFT', 'snatch_first_weight': 95, 'snatch_second': 'LIFT', 'snatch_second_weight': 98, 'snatch_third': 'LIFT', 'snatch_third_weight': 100, 'cnj_first': 'LIFT', 'cnj_first_weight': 115, 'cnj_second': 'LIFT', 'cnj_second_weight': 118, 'cnj_third': 'LIFT', 'cnj_third_weight': 122, 'bodyweight': 86.0, 'weight_category': 'W87', 'team': 'NZ', 'lottery_number': 7}
QZN7Qrv
Lift created successfully


598


In [87]:
start = 598
interval = 20
session_id = "arDNGgQ"

create_athlete_lift(
    start=start,
    interval=interval, 
    table_text=table_text,
    competition_id=competition_id,
    session_id=session_id
    )

['6'] ['9'] ['Whiting'] ['Hayley'] ['F 87'] ['83.10'] ['New', 'Zealand'] ['1993'] ['95'] ['(98)'] ['98'] ['98'] ['116'] ['120'] ['125'] ['125'] ['223'] ['1'] ['253.593'] ['399.234'] athlete created Hayley Whiting
{'athlete_name': 'Hayley Whiting', 'snatch_first': 'LIFT', 'snatch_first_weight': 95, 'snatch_second': 'NOLIFT', 'snatch_second_weight': 98, 'snatch_third': 'LIFT', 'snatch_third_weight': 98, 'cnj_first': 'LIFT', 'cnj_first_weight': 116, 'cnj_second': 'LIFT', 'cnj_second_weight': 120, 'cnj_third': 'LIFT', 'cnj_third_weight': 125, 'bodyweight': 83.1, 'weight_category': 'W87', 'team': 'NZ', 'lottery_number': 9}
gdrPoYp
Lift created successfully


618


In [92]:
start = 619
interval = 20
session_id = "arDNGgQ"

create_athlete_lift(
    start=start,
    interval=interval, 
    table_text=table_text,
    competition_id=competition_id,
    session_id=session_id
    )

['7'] ['2'] ['Summerell'] ['Miniah'] ['F>87'] ['111.30'] ['New', 'Zealand'] ['2003'] ['(80)'] ['83'] ['(87)'] ['83'] ['100'] ['(105)'] ['109'] ['109'] ['192'] ['2'] ['198.916'] ['162.152'] athlete already exists
{'athlete_name': 'Miniah Summerell', 'snatch_first': 'NOLIFT', 'snatch_first_weight': 80, 'snatch_second': 'LIFT', 'snatch_second_weight': 83, 'snatch_third': 'NOLIFT', 'snatch_third_weight': 87, 'cnj_first': 'LIFT', 'cnj_first_weight': 100, 'cnj_second': 'NOLIFT', 'cnj_second_weight': 105, 'cnj_third': 'LIFT', 'cnj_third_weight': 109, 'bodyweight': 111.3, 'weight_category': 'W87+', 'team': 'NZ', 'lottery_number': 2}
nBAl1Nz
Lift created successfully


639


In [93]:
start = 639
interval = 20
session_id = "arDNGgQ"

create_athlete_lift(
    start=start,
    interval=interval, 
    table_text=table_text,
    competition_id=competition_id,
    session_id=session_id
    )

['8'] ['29'] ['Edwards'] ['Ramsi'] ['F>87'] ['105.90'] ['Niue'] ['2006'] ['50'] ['57'] ['62'] ['62'] ['70'] ['75'] ['(80)'] ['75'] ['137'] ['3'] ['143.613'] ['126.963'] athlete created Ramsi Edwards
{'athlete_name': 'Ramsi Edwards', 'snatch_first': 'LIFT', 'snatch_first_weight': 50, 'snatch_second': 'LIFT', 'snatch_second_weight': 57, 'snatch_third': 'LIFT', 'snatch_third_weight': 62, 'cnj_first': 'LIFT', 'cnj_first_weight': 70, 'cnj_second': 'LIFT', 'cnj_second_weight': 75, 'cnj_third': 'NOLIFT', 'cnj_third_weight': 80, 'bodyweight': 105.9, 'weight_category': 'W87+', 'team': 'NU', 'lottery_number': 29}
3dAOgAg
Lift created successfully


659


In [84]:
start = 659
interval = 20
session_id = "arDNGgQ"

create_athlete_lift(
    start=start,
    interval=interval, 
    table_text=table_text,
    competition_id=competition_id,
    session_id=session_id
    )

['9'] ['34'] ['Lynch'] ['Manine'] ['F>87'] ['119.90'] ['Cook', 'Islands'] ['1999'] ['82'] ['86'] ['(90)'] ['86'] ['105'] ['(110)'] ['110'] ['110'] ['196'] ['1'] ['200.147'] ['177.169'] 

679


In [101]:
# had to enter this manually??

start = 705
interval = 20
session_id = "7agrADQ"

create_athlete_lift(
    start=start,
    interval=interval, 
    table_text=table_text,
    competition_id=competition_id,
    session_id=session_id
    )

['1'] ['6'] ['Pomelile'] ['Joe'] ['M', '109'] ['108.90'] ['Tonga'] ['1993'] ['(135)'] ['135'] ['140'] ['140'] ['(173)'] ['(173)'] ['174'] ['174'] ['314'] ['3'] ['338.247'] ['338.649'] athlete already exists
{'athlete_name': 'Joe Pomelile', 'snatch_first': 'NOLIFT', 'snatch_first_weight': 135, 'snatch_second': 'LIFT', 'snatch_second_weight': 135, 'snatch_third': 'LIFT', 'snatch_third_weight': 140, 'cnj_first': 'NOLIFT', 'cnj_first_weight': 173, 'cnj_second': 'NOLIFT', 'cnj_second_weight': 173, 'cnj_third': 'LIFT', 'cnj_third_weight': 174, 'bodyweight': 108.9, 'weight_category': 'M109', 'team': 'TO', 'lottery_number': 6}
J5r95r6


NotAllowedError: status code returned: 400

In [103]:
start = 725
interval = 20
session_id = "7agrADQ"

create_athlete_lift(
    start=start,
    interval=interval, 
    table_text=table_text,
    competition_id=competition_id,
    session_id=session_id
    )

['2'] ['12'] ['Jordan'] ['Robert'] ['M', '109'] ['101.00'] ['Guest'] ['1986'] ['125'] ['128'] ['(136)'] ['128'] ['155'] ['162'] ['(170)'] ['162'] ['290'] ['4'] ['320.406'] ['311.465'] athlete created Robert Jordan
{'athlete_name': 'Robert Jordan', 'snatch_first': 'LIFT', 'snatch_first_weight': 125, 'snatch_second': 'LIFT', 'snatch_second_weight': 128, 'snatch_third': 'NOLIFT', 'snatch_third_weight': 136, 'cnj_first': 'LIFT', 'cnj_first_weight': 155, 'cnj_second': 'LIFT', 'cnj_second_weight': 162, 'cnj_third': 'NOLIFT', 'cnj_third_weight': 170, 'bodyweight': 101.0, 'weight_category': 'M109', 'team': 'NZ', 'lottery_number': 12}
5gNm3ra
Lift created successfully


745


In [102]:
start = 745
interval = 20
session_id = "7agrADQ"

create_athlete_lift(
    start=start,
    interval=interval,
    table_text=table_text,
    competition_id=competition_id,
    session_id=session_id
    )

['3'] ['13'] ['Tasi', 'Taala'] ['Koale', 'Junior'] ['M', '109'] ['109.00'] ['New', 'Zealand'] ['1995'] ['135'] ['143'] ['(155)'] ['143'] ['178'] ['185'] ['192'] ['192'] ['335'] ['1'] ['360.765'] ['419.900'] athlete created Koale Tasi
{'athlete_name': 'Koale Tasi', 'snatch_first': 'LIFT', 'snatch_first_weight': 135, 'snatch_second': 'LIFT', 'snatch_second_weight': 143, 'snatch_third': 'NOLIFT', 'snatch_third_weight': 155, 'cnj_first': 'LIFT', 'cnj_first_weight': 178, 'cnj_second': 'LIFT', 'cnj_second_weight': 185, 'cnj_third': 'LIFT', 'cnj_third_weight': 192, 'bodyweight': 109.0, 'weight_category': 'M109', 'team': 'NZ', 'lottery_number': 13}
RwNzWYL
Lift created successfully


765


In [104]:
start = 765
interval = 20
session_id = "7agrADQ"

create_athlete_lift(
    start=start,
    interval=interval, 
    table_text=table_text,
    competition_id=competition_id,
    session_id=session_id
    )

['4'] ['20'] ['Vakuruivalu'] ['Timothy'] ['M', '109'] ['102.60'] ['Fiji'] ['1999'] ['110'] ['115'] ['(118)'] ['115'] ['135'] ['(140)'] ['(143)'] ['135'] ['250'] ['5'] ['274.672'] ['158.823'] athlete created Timothy Vakuruivalu
{'athlete_name': 'Timothy Vakuruivalu', 'snatch_first': 'LIFT', 'snatch_first_weight': 110, 'snatch_second': 'LIFT', 'snatch_second_weight': 115, 'snatch_third': 'NOLIFT', 'snatch_third_weight': 118, 'cnj_first': 'LIFT', 'cnj_first_weight': 135, 'cnj_second': 'NOLIFT', 'cnj_second_weight': 140, 'cnj_third': 'NOLIFT', 'cnj_third_weight': 143, 'bodyweight': 102.6, 'weight_category': 'M109', 'team': 'NZ', 'lottery_number': 20}
aWAkmY1
Lift created successfully


785


In [107]:
start = 785
interval = 20
session_id = "7agrADQ"

create_athlete_lift(
    start=start,
    interval=interval, 
    table_text=table_text,
    competition_id=competition_id,
    session_id=session_id
    )

['5'] ['26'] ['Leuo'] ['Siaosi'] ['M', '109'] ['108.80'] ['New', 'Zealand'] ['1992'] ['130'] ['135'] ['(140)'] ['135'] ['(180)'] ['181'] ['(186)'] ['181'] ['316'] ['2'] ['340.499'] ['345.867'] athlete created Siaosi Leuo
{'athlete_name': 'Siaosi Leuo', 'snatch_first': 'LIFT', 'snatch_first_weight': 130, 'snatch_second': 'LIFT', 'snatch_second_weight': 135, 'snatch_third': 'NOLIFT', 'snatch_third_weight': 140, 'cnj_first': 'NOLIFT', 'cnj_first_weight': 180, 'cnj_second': 'LIFT', 'cnj_second_weight': 181, 'cnj_third': 'NOLIFT', 'cnj_third_weight': 186, 'bodyweight': 108.8, 'weight_category': 'M109', 'team': 'NZ', 'lottery_number': 26}
zYxVnAG
Lift created successfully


805


In [109]:
start = 806
interval = 20
session_id = "7agrADQ"

create_athlete_lift(
    start=start,
    interval=interval, 
    table_text=table_text,
    competition_id=competition_id,
    session_id=session_id
    )

['6'] ['14'] ['Toimata'] ['Gio'] ['M>109'] ['112.90'] ['Niue'] ['1992'] ['100'] ['105'] ['110'] ['110'] ['130'] ['140'] ['146'] ['146'] ['256'] ['2'] ['272.800'] ['120.542'] athlete created Gio Toimata
{'athlete_name': 'Gio Toimata', 'snatch_first': 'LIFT', 'snatch_first_weight': 100, 'snatch_second': 'LIFT', 'snatch_second_weight': 105, 'snatch_third': 'LIFT', 'snatch_third_weight': 110, 'cnj_first': 'LIFT', 'cnj_first_weight': 130, 'cnj_second': 'LIFT', 'cnj_second_weight': 140, 'cnj_third': 'LIFT', 'cnj_third_weight': 146, 'bodyweight': 112.9, 'weight_category': 'M109+', 'team': 'NU', 'lottery_number': 14}
gA5xxr0
Lift created successfully


826


In [110]:
start = 826
interval = 20
session_id = "7agrADQ"

create_athlete_lift(
    start=start,
    interval=interval, 
    table_text=table_text,
    competition_id=competition_id,
    session_id=session_id
    )

['7'] ['22'] ['Faamausili-Fala'] ['Malachi'] ['M >109'] ['130.60'] ['New', 'Zealand'] ['2001'] ['(150)'] ['150'] ['(155)'] ['150'] ['(170)'] ['171'] ['180'] ['180'] ['330'] ['1'] ['339.549'] ['280.194'] athlete created Malachi Faamausili-Fala
{'athlete_name': 'Malachi Faamausili-Fala', 'snatch_first': 'NOLIFT', 'snatch_first_weight': 150, 'snatch_second': 'LIFT', 'snatch_second_weight': 150, 'snatch_third': 'NOLIFT', 'snatch_third_weight': 155, 'cnj_first': 'NOLIFT', 'cnj_first_weight': 170, 'cnj_second': 'LIFT', 'cnj_second_weight': 171, 'cnj_third': 'LIFT', 'cnj_third_weight': 180, 'bodyweight': 130.6, 'weight_category': 'M109+', 'team': 'NZ', 'lottery_number': 22}
vrWL2Ap
Lift created successfully


846


In [111]:
start = 846
interval = 20
session_id = "7agrADQ"

create_athlete_lift(
    start=start,
    interval=interval, 
    table_text=table_text,
    competition_id=competition_id,
    session_id=session_id
    )



866


In [ ]:
# END